<a href="https://colab.research.google.com/github/vortexash/Tensorflow-Developer/blob/main/06_transfer_learning_in_tensorflow_part_3_scaling_up_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning with Tensorflow Part 3: Scaling up (Food Vision mini)

We've seen the power of transfer Learning feature extraction and fine-tuning, now it;s time to scale up to all the classes in Food101 (101 total classes of food)

Our goal is to beat the original Food101 papre with 10% of the training data (leveraging the power of Deep Learning)

Original Food101 paper: https://data.vision.ee.ethz.ch/cvl/datasets_extra/food-101/static/bossard_eccv14_food-101.pdf

Our basekine to beat is 50.76% across 101 classes.


In [1]:
# Check to see if we're using a GPU
!nvidia-smi

Sun Jan 16 17:26:11 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Creating helper functions

In previous notebooks, we've created a series of healper functions to do different tasks, let's download them.

In [2]:
!wget https://raw.githubusercontent.com/vortexash/tensorflow-deep-learning/main/extras/helper_functions.py

--2022-01-16 17:28:48--  https://raw.githubusercontent.com/vortexash/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-01-16 17:28:48 (60.0 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [4]:
# Import series of helper functions for our notebook.
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, walk_through_dir, compare_historys

## 101 Food  Classes: woring with less data

Our goal is to beat the original Food101 paper with 10% of the training data, so Let's download it.

The data we're downloading comes from the original Food101  data set but has been preprocessed using image modification notebook.


In [5]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip


--2022-01-16 17:33:53--  https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.5.128, 64.233.184.128, 64.233.166.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.5.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1625420029 (1.5G) [application/zip]
Saving to: ‘101_food_classes_10_percent.zip’

101_food_classes_10 100%[===================>]   1.51G   103MB/s    in 16s     

2022-01-16 17:34:10 (97.8 MB/s) - ‘101_food_classes_10_percent.zip’ saved [1625420029/1625420029]



In [11]:
unzip_data("101_food_classes_10_percent.zip")
train_dir = "/content/101_food_classes_10_percent/train"
test_dir = "/content/101_food_classes_10_percent/test"

In [7]:
# How many images/classes are there?
walk_through_dir("101_food_classes_10_percent.zip")

In [10]:
# Setup data inputs
import tensorflow as tf
IMG_SIZE =(224,224)
train_data_all_10_percent = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                           label_mode="categorical",
                                                                           image_size=IMG_SIZE)
test_data_all_10_percent = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                           label_mode="categorical",
                                                                           image_size=IMG_SIZE,
                                                                          shuffle=False) # don't shuffle test data fro prediction analysis

NotADirectoryError: ignored